# 프로젝트 6: 멋진 작사가 만들기

**라이브러리 버전 확인하기**

In [3]:
import tensorflow

print(tensorflow.__version__)

2.8.2


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**데이터 다운로드**

In [5]:
!mkdir -p ~/drive/lyricist/models
!ln -s ~/data ~/drive/lyricist/data

**데이터 읽어오기**

In [6]:
import glob
import os, re
import numpy as np
import tensorflow as tf

txt_file_path = '/content/drive/MyDrive/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus에 담는다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['Looking for some education', 'Made my way into the night', 'All that bullshit conversation']


In [7]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue  # 길이가 0인 문장은 건너뛴다.
    
    if idx > 9: break  # 문장 10개 확인

    print(sentence)

Looking for some education
Made my way into the night
All that bullshit conversation
Baby, can't you read the signs? I won't bore you with the details, baby
I don't even wanna waste your time
Let's just say that maybe
You could help me ease my mind
I ain't Mr. Right But if you're looking for fast love
If that's love in your eyes
It's more than enough


**데이터 정체**

In [8]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2. 특수문자 양쪽에 공백을 넣고
    sentence = re.sub(r'[" "]+', " ", sentence) # 3. 여러개의 공백은 하나의 공백으로 바꿉니다
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
    sentence = sentence.strip() # 5. 다시 양쪽 공백을 지웁니다
    sentence = '<start> ' + sentence + ' <end>' # 6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
    return sentence

In [9]:
# 여기에 정제된 문장을 모은다.
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뛴다.
    if len(sentence) == 0: continue
    if len(sentence.split()) >= 13: continue  # 토큰의 개수 15개 넘어가는 문장 제외(start,end포함)
    
    # 정제를 하고 담는다.
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보자
corpus[:10]

['<start> looking for some education <end>',
 '<start> made my way into the night <end>',
 '<start> all that bullshit conversation <end>',
 '<start> i don t even wanna waste your time <end>',
 '<start> let s just say that maybe <end>',
 '<start> you could help me ease my mind <end>',
 '<start> i ain t mr . right but if you re looking for fast love <end>',
 '<start> if that s love in your eyes <end>',
 '<start> it s more than enough <end>',
 '<start> had some bad love <end>']

**평가 데이터셋 분리**

In [10]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)   
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[  2 290  28 ...   0   0   0]
 [  2 223  13 ...   0   0   0]
 [  2  25  17 ...   0   0   0]
 ...
 [  2  22  76 ...   0   0   0]
 [  2  44  26 ...   0   0   0]
 [  2  22  76 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7f5423c1a750>


In [11]:
print(tensor[:3,:10])

[[   2  290   28   96 4536    3    0    0    0    0]
 [   2  223   13   87  224    6  115    3    0    0]
 [   2   25   17 1072 2290    3    0    0    0    0]]


In [12]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


In [13]:
from sklearn.model_selection import train_test_split

src_input = tensor[:, :-1]
tgt_input = tensor[:,1:]

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input,test_size=0.2)

print(src_input[0])
print(tgt_input[0])

[   2  290   28   96 4536    3    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0]
[ 290   28   96 4536    3    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0]


In [14]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

# tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 12001개
VOCAB_SIZE = tokenizer.num_words + 1

# 준비한 데이터 소스로부터 데이터셋을 만든다.
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder = True)
dataset

<BatchDataset element_spec=(TensorSpec(shape=(256, 32), dtype=tf.int32, name=None), TensorSpec(shape=(256, 32), dtype=tf.int32, name=None))>

**인공지능 만들기**

In [15]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [16]:
for src_sample, tgt_sample in dataset.take(1): break

model(src_sample)

<tf.Tensor: shape=(256, 32, 12001), dtype=float32, numpy=
array([[[ 8.36752224e-06, -1.59816063e-05,  1.77276364e-04, ...,
          9.05434063e-05,  2.39563000e-04,  2.92930315e-04],
        [ 5.42002344e-05,  7.83055366e-05,  5.00587979e-04, ...,
          4.63830162e-04,  3.94190225e-04,  3.99739161e-04],
        [ 3.44010943e-04,  3.02380853e-04,  3.99545184e-04, ...,
          6.49386551e-04,  3.95234325e-04,  4.34430374e-04],
        ...,
        [-2.90568452e-03, -6.07808412e-04, -1.55156036e-03, ...,
         -2.94283032e-03,  4.43194387e-03,  2.48325232e-04],
        [-2.92123039e-03, -6.19547849e-04, -1.59434450e-03, ...,
         -2.97844573e-03,  4.47222777e-03,  1.86859033e-04],
        [-2.92983232e-03, -6.33107265e-04, -1.62824348e-03, ...,
         -3.00522242e-03,  4.50121379e-03,  1.25784529e-04]],

       [[ 8.36752224e-06, -1.59816063e-05,  1.77276364e-04, ...,
          9.05434063e-05,  2.39563000e-04,  2.92930315e-04],
        [-3.70222842e-05, -1.83567710e-04,  3

In [17]:
model.summary()

Model: "text_generator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  3072256   
                                                                 
 lstm (LSTM)                 multiple                  5246976   
                                                                 
 lstm_1 (LSTM)               multiple                  8392704   
                                                                 
 dense (Dense)               multiple                  12301025  
                                                                 
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


In [19]:
import tensorflow as tf
from keras.callbacks import EarlyStopping
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
history = model.fit(enc_train,dec_train, validation_data=(enc_val, dec_val), epochs = 10, batch_size = 500)

Epoch 1/10
255/255 [==============================] - 108s 399ms/step - loss: 0.8751 - val_loss: 0.9061
Epoch 2/10
255/255 [==============================] - 101s 397ms/step - loss: 0.8380 - val_loss: 0.9157
Epoch 3/10
255/255 [==============================] - 103s 404ms/step - loss: 0.8162 - val_loss: 0.9210
Epoch 4/10
255/255 [==============================] - 103s 404ms/step - loss: 0.7965 - val_loss: 0.9280
Epoch 5/10
255/255 [==============================] - 101s 398ms/step - loss: 0.7779 - val_loss: 0.9314
Epoch 6/10
255/255 [==============================] - 101s 397ms/step - loss: 0.7599 - val_loss: 0.9365
Epoch 7/10
255/255 [==============================] - 103s 404ms/step - loss: 0.7424 - val_loss: 0.9396
Epoch 8/10
255/255 [==============================] - 103s 404ms/step - loss: 0.7255 - val_loss: 0.9417
Epoch 9/10
255/255 [==============================] - 103s 404ms/step - loss: 0.7092 - val_loss: 0.9465
Epoch 10/10
255/255 [==============================] - 101s 398m

In [36]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환한다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    while True:
        predict = model(test_tensor)  # 1. 입력받은 문장의 텐서를 입력한다.
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1),axis=-1)[:,-1]   # 2. 예측된 값 중 가장 높은 확률인 word index를 뽑아낸다.
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)   # 3. 2에서 예측된 word index를 문장 뒤에 붙인다.
        if predict_word.numpy()[0] == end_token: break   # 4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마친다.
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    for word_index in test_tensor[0].numpy():  # tokenizer를 이용해 word index를 단어로 하나씩 변환한다. 
        generated += tokenizer.index_word[word_index] + " "
    
    return generated

In [37]:
# Loss
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits = True, reduction='none')

In [38]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you , i m not gonna crack <end> '

난 널 사랑해, 난 깨지지 않을 거야

**✍ 회고**

* Loss는 모델이 오답을 만들고 있는 정도라고 생각하면 된다. 출력된 loss를 보면, 매우 준수하게 오답을 만들고 있다고 판단이 된다. 해당 프로젝트에서 val_loss 값을 2.2 수준으로 줄이라고 했는데, 출력된 val_loss가 대체적으로 1을 넘지 않았다. 추가로 val_loss는 검증데이터에 대한 손실을 나타낸다.

* 모델이 출력한 가사는 i love you , i m not gonna crack였다. 의미는 '난 널 사랑해, 난 깨지지 않을 거야'이다. 신기한 멘트를 알아서 출력하는 게 재밌다. 배우면 배울수록 신기하고 새롭다...

* val_loss를 출력하기 위해 history = model.fit(enc_train,dec_train, validation_data=(enc_val, dec_val), epochs = 10, batch_size = 500) 을 추가해줬다. 적당히 순조로웠다.

* epoch를 10번 돌리는데 시간이 좀 걸렸다. 기다리는 재미도 있고, 값 나오는 재미도 있고, 넘 즐거웠다...!